In [18]:
import gurobipy as gp
from gurobipy import GRB

TASKS = {
    ('job_1', 'machine_1'): {'dur': 10, 'prec': None},
    ('job_1', 'machine_2'): {'dur': 8, 'prec': ('job_1', 'machine_1')},
    ('job_1', 'machine_3'): {'dur': 4, 'prec': ('job_1', 'machine_2')},
    ('job_2', 'machine_2'): {'dur': 8, 'prec': None},
    ('job_2', 'machine_1'): {'dur': 3, 'prec': ('job_2', 'machine_2')},
    ('job_2', 'machine_4'): {'dur': 5, 'prec': ('job_2', 'machine_1')},
    ('job_2', 'machine_3'): {'dur': 6, 'prec': ('job_2', 'machine_4')},
    ('job_3', 'machine_1'): {'dur': 4, 'prec': None},
    ('job_3', 'machine_2'): {'dur': 7, 'prec': ('job_3', 'machine_1')},
    ('job_3', 'machine_4'): {'dur': 3, 'prec': ('job_3', 'machine_2')},
}

def jobshop_model_gurobi(TASKS):
    model = gp.Model()

    TASKS_set = set(TASKS.keys())
    JOBS = {j for (j, m) in TASKS_set}
    MACHINES = {m for (j, m) in TASKS_set}

    TASKORDER_set = {(j, m, k, n) for (j, m) in TASKS_set for (k, n) in TASKS_set if (k, n) == TASKS[(j, m)]['prec']}
    DISJUNCTIONS_set = {(j, k, m) for j in JOBS for k in JOBS if j < k for m in MACHINES if (j, m) in TASKS_set and (k, m) in TASKS_set}

    dur = {(j, m): TASKS[(j, m)]['dur'] for (j, m) in TASKS_set}

    ub = sum(dur[jm] for jm in TASKS_set)

    makespan = model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=ub, name="makespan")
    start = {(j, m): model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=ub, name=f"start_{j}_{m}") for (j, m) in TASKS_set}

    model.setObjective(makespan, GRB.MINIMIZE)

    finish_constr = {jm: model.addConstr(start[jm] + dur[jm] <= makespan, name=f"finish_{jm[0]}_{jm[1]}") for jm in TASKS_set}
    preceding_constr = {task_order: model.addConstr(start[task_order[2], task_order[3]] + dur[task_order[2], task_order[3]] <= start[task_order[0], task_order[1]], name=f"preceding_{task_order[0]}_{task_order[1]}_{task_order[2]}_{task_order[3]}") for task_order in TASKORDER_set}
    disjunctions_constr = {(j, k, m): [
        model.addConstr(start[j, m] + dur[j, m] <= start[k, m], name=f"disj_{j}_{k}_{m}"),
        model.addConstr(start[k, m] + dur[k, m] <= start[j, m], name=f"disj_{k}_{j}_{m}")]
        for (j, k, m) in DISJUNCTIONS_set}

    model.optimize()

    return model

model=jobshop_model_gurobi(TASKS)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Pentium(R) CPU 4425Y @ 1.70GHz, instruction set [SSE2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 33 rows, 11 columns and 66 nonzeros
Model fingerprint: 0xfba13b88
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [6e+01, 6e+01]
  RHS range        [3e+00, 1e+01]
Presolve time: 0.01s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Infeasible model


In [20]:
model.status == gp.GRB.OPTIMAL

False